In [14]:
import pandas as pd
from itertools import combinations
import math
import os
import csv
import time

## Reads input data

In [15]:
demand = pd.read_csv('input_data/platforms_demand_18.csv', header=0, delimiter=';')
distances = pd.read_csv('input_data/distance_matrix_18.csv', header=0, delimiter=';')

## Algorithm for generating the shortest routes

In [16]:
start_time = time.time()

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['Mon'] + demand['platform'].tolist() + ['Mon']  # Add 'Mon' as start and end platforms

distances_matrix = distances.set_index('from/to').map(lambda x: float(x.replace(',', '.')))

shortest_routes_dict = {}

max_platforms = 7 
cargo_capacity = 100 

for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):  # Start from 3 and add 3 for 'Mon' at the beginning and end
    for route_combination in combinations(platforms_demand.keys(), r - 2):  # Subtract 2 for 'Mon' at the beginning and end
        route = ['Mon'] + list(route_combination) + ['Mon']
        total_distance = 0
        total_demand = 0
        duration_lossing = 0
        valid_route = True
        
        for i in range(len(route) - 1):
            from_platform = platforms_d.index(route[i])
            to_platform = platforms_d.index(route[i + 1])
            total_distance += distances_matrix.iloc[from_platform, to_platform]
            if route[i] != 'Mon':  # Exclude 'Mon' from demand check
                total_demand += platforms_demand[route[i]]
                if total_demand > cargo_capacity:
                    valid_route = False
                    break
        
        if valid_route:
            total_distance = round(total_distance, 1)
            duration_sailing = total_distance / 10
            duration_lossing = (total_demand * 1.389) / 10
            duration_sailing = round(duration_sailing, 2)
            duration_lossing = round(duration_lossing, 2)
            
            # Create a key by sorting the visited platforms to handle different orders
            key = tuple(sorted(set(route)))
            
            # Check if this route dominates any existing route for the same set of platforms
            dominated = False
            if key in shortest_routes_dict:
                existing_route_distance = shortest_routes_dict[key][1]
                if total_distance < existing_route_distance:
                    dominated = True
                elif total_distance == existing_route_distance:
                    # If distances are equal, prioritize the one with less demand
                    existing_route_demand = shortest_routes_dict[key][2]
                    if total_demand >= existing_route_demand:
                        dominated = True
                    else:
                        # Update the existing route with the new route information
                        shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            
            # If the new route does not dominate any existing route, add it to the dictionary
            if not dominated:
                shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {distance}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print("Running time", time.time() - start_time)

Shortest Route: ['Mon', 'APT', 'Mon'], Total Distance: 134.2, Total Demand: 18.0, Duration sailing: 13.42, Duration lossing: 2.5
Shortest Route: ['Mon', 'ASL', 'Mon'], Total Distance: 159.8, Total Demand: 25.0, Duration sailing: 15.98, Duration lossing: 3.47
Shortest Route: ['Mon', 'DAB', 'Mon'], Total Distance: 92.0, Total Demand: 40.0, Duration sailing: 9.2, Duration lossing: 5.56
Shortest Route: ['Mon', 'DSA', 'Mon'], Total Distance: 175.0, Total Demand: 29.0, Duration sailing: 17.5, Duration lossing: 4.03
Shortest Route: ['Mon', 'DSS', 'Mon'], Total Distance: 170.6, Total Demand: 29.0, Duration sailing: 17.06, Duration lossing: 4.03
Shortest Route: ['Mon', 'GF', 'Mon'], Total Distance: 156.2, Total Demand: 68.0, Duration sailing: 15.62, Duration lossing: 9.45
Shortest Route: ['Mon', 'KVB', 'Mon'], Total Distance: 131.4, Total Demand: 14.0, Duration sailing: 13.14, Duration lossing: 1.94
Shortest Route: ['Mon', 'MID', 'Mon'], Total Distance: 164.2, Total Demand: 14.0, Duration saili

In [17]:
print(len(shortest_routes_dict))

3121


## Checks the capacity utilization (demand delivered). Removes all routes with capacity utilization below 70%

In [18]:
count_below_10 = 0
count_below_20 = 0
count_below_30 = 0
count_below_40 = 0
count_below_50 = 0
count_below_60 = 0
count_below_70 = 0
count_below_80 = 0
count_below_90 = 0

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    if demand < 10:
        count_below_10 += 1
    if demand < 20:
        count_below_20 += 1
    if demand < 30:
        count_below_30 += 1
    if demand < 40:
        count_below_40 += 1
    if demand < 50:
        count_below_50 += 1
    if demand < 60:
        count_below_60 += 1
    if demand < 70:
        count_below_70 += 1
    if demand < 80:
        count_below_80 += 1
    if demand < 90:
        count_below_90 += 1

print("Number of shortest routes with total demand below 10:", count_below_10)
print("Number of shortest routes with total demand below 20:", count_below_20)
print("Number of shortest routes with total demand below 30:", count_below_30)
print("Number of shortest routes with total demand below 40:", count_below_40)
print("Number of shortest routes with total demand below 50:", count_below_50)
print("Number of shortest routes with total demand below 60:", count_below_60)
print("Number of shortest routes with total demand below 70:", count_below_70)
print("Number of shortest routes with total demand below 80:", count_below_80)
print("Number of shortest routes with total demand below 90:", count_below_90)

Number of shortest routes with total demand below 10: 0
Number of shortest routes with total demand below 20: 8
Number of shortest routes with total demand below 30: 24
Number of shortest routes with total demand below 40: 55
Number of shortest routes with total demand below 50: 131
Number of shortest routes with total demand below 60: 296
Number of shortest routes with total demand below 70: 569
Number of shortest routes with total demand below 80: 1080
Number of shortest routes with total demand below 90: 1794


In [19]:
shortest_routes_dict_cap = {}

for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict.items():
    if demand > 70:
        shortest_routes_dict_cap[key] = (route, distance, demand, duration_sailing, duration_lossing)

In [20]:
print(len(shortest_routes_dict_cap))

2508


## Removes routes which results in very long idle time

In [21]:
count_between_16_and_24 = 0
count_between_40_and_48 = 0

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict_cap.values():
    total_duration = duration_sailing + duration_lossing
    if 16 < total_duration < 24:
        count_between_16_and_24 += 1
    if 40 < total_duration < 48:
        count_between_40_and_48 += 1

print("Count of routes with duration between 16 and 24:", count_between_16_and_24)
print("Count of routes with duration between 40 and 48:", count_between_40_and_48)

Count of routes with duration between 16 and 24: 7
Count of routes with duration between 40 and 48: 207


In [22]:
shortest_routes_dict_cap_idle = {}
routes_to_remove = []

for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict_cap.items():
    total_duration = duration_sailing + duration_lossing
    if 16 < total_duration < 24 or 40 < total_duration < 48:
        routes_to_remove.append(key)
    else:
        shortest_routes_dict_cap_idle[key] = (route, distance, demand, duration_sailing, duration_lossing)

for key in routes_to_remove:
    del shortest_routes_dict_cap[key]

In [23]:
print(len(shortest_routes_dict_cap))

2294


## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 2294 routes

In [24]:
routes_only = [route for route, _, _, _, _ in shortest_routes_dict_cap.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict_cap.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict_cap.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict_cap.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict_cap.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv('generated_datafiles/distances.csv', index=False)
df_demand.to_csv('generated_datafiles/demand.csv', index=False)
df_duration_sailing.to_csv('generated_datafiles/duration_sailing.csv', index=False)
df_duration_lossing.to_csv('generated_datafiles/duration_lossing.csv', index=False)

route_file = "generated_datafiles/routes.csv"

def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)